*UE Learning from User-generated Data, CP MMS, JKU Linz 2024*
# Exercise 5: Popularity bias evaluation

In this exercise we evaluate popularity bias of three different RecSys we already implemented. Note that here we utilize only one possible approach to evaluation of popularity bias from the user perspective. Refer to the slides and the recording for more details.

This time recommendations for each algorithm are already provided in three separate files (can be found on the main page).

Evaluating popularity bias through the mismatch (miscalibration) between popularity distributions over users' listening histories and their respective recommendations requires knowing how popular each item of the dataset is. Please use the new version of the lfm-tiny-tunes dataset where popularity categories (`popularity_bins`) from `0` (LowPop) to `2` (HighPop) have been already assigned. You will not need the test set for this exercise.

Make sure to rename the notebook according to the convention:

LUD24_ex06_k<font color='red'><Matr. Number\></font>_<font color='red'><Surname-Name\></font>.ipynb

for example:

LUD24_ex06_k000007_Bond_James.ipynb

## Implementation
In this exercise, as before, you are required to write a number of functions. Only implemented functions are graded. Insert your implementations into the templates provided. Please don't change the templates even if they are not pretty. Don't forget to test your implementation for correctness and efficiency. **Make sure to try your implementations on toy examples and sanity checks.**

Please **only use libraries already imported in the notebook**.

In [14]:
import numpy as np
import pandas as pd
from rec import inter_matr_implicit

## Task 1

Implement Kullback–Leibler divergence (KL) and Jensen–Shannon divergence (JSD).

#### in KL
Make sure to
* use the logarithm with base 2
* add `eps` to both `P` and `Q` distributions
* **normalize** (convert to probability) the distributions again after you have added `eps`

#### for JSD
* feel free to call your implementation of KL from it
* refer to the recording and slides for a recap

In [15]:
def KL(P: np.ndarray, Q: np.ndarray, eps: float = 1e-14):
    '''
    P - 1D np.ndarray, probability distribution;
    Q - 1D np.ndarray, probability distribution;
    eps - float, margin added to both distributions to avoid division by zero errors;

    returns - float, divergence of distributions P,Q;
    '''

    P = P + eps
    Q = Q + eps

    #print("P + eps:", P)
    #print("Q + eps:", Q)
    
    P = P / np.sum(P)
    Q = Q / np.sum(Q)

    #print("Norma P:", P)
    #print("Norma Q:", Q)
    
    kl = np.sum(P * np.log2(P / Q))

    #print("KL diver:", kl)
    
    return float(kl)


In [16]:
# Check if you handle zeros correctly (use "eps" parameter)
P = np.array([0.3, 0.6, 0.1])
Q = np.array([0.4, 0.6, 0.0])

assert type(KL(P, Q)) == float
assert np.isclose(KL(P, Q), 4.193995)

In [17]:
def JSD(P: np.ndarray, Q: np.ndarray, eps: float = 1e-14):
    '''
    P - 1D np.ndarray, probability distribution;
    Q - 1D np.ndarray, probability distribution;
    eps - float, margin added to both distributions to avoid division by zero errors, parameter for KL;

    returns - float, divergence of distributions P,Q;
    '''
    M = 0.5 * (P + Q)

    #print("Avg distr M:", M)
    
    jsd = 0.5 * KL(P, M, eps) + 0.5 * KL(Q, M, eps)

    #print("JSD:", jsd)
    
    return float(jsd)

In [18]:
# Check if you handle zeros correctly (use "eps" parameter)
P = np.array([0.3, 0.6, 0.1])
Q = np.array([0.4, 0.6, 0.0])

assert type(JSD(P, Q)) == float
assert np.isclose(JSD(P, Q), 0.055170)

In [19]:
# KL divergence compares a target distribution to a reference distribution and is NOT symmetric
# JS divergence is symmetric -  you can exchange P and Q and get the same results
# the following asserts check this behaviour
P = np.array([0.3, 0.6, 0.1])
Q = np.array([0.2, 0.3, 0.5])

assert KL(P, Q) != KL(Q, P)
assert JSD(P, Q) == JSD(Q, P)

## Task 2

Evaluating algorithms for popularity bias.
First, make sure all the data is loaded correctly.

In [20]:
def read(dataset, file):
    return pd.read_csv(dataset + '/' + dataset + '.' + file, sep='\t')


items = read("lfm-tiny-tunes", 'item')
users = read("lfm-tiny-tunes", 'user')
train_inters = read("lfm-tiny-tunes", 'inter_train')

train_inter_matrix = inter_matr_implicit(users=users, items=items, interactions=train_inters, dataset_name="lfm-tiny-tunes")

# Information about item popularity: 0 - LowPop, 2 - HighPop
item_pop_bins = items["popularity_bins"].to_numpy()
# Train set - users' listening history!
train_interaction_matrix = train_inter_matrix

In [21]:
config = {
    "item_pop_bins": item_pop_bins,
    "user_histories": train_interaction_matrix,
    "recommenders": {}  # here you can access the recommendations for every algorithm
}

recommendations = {"recommenders": {
    "SVD": {
        "recommendations": np.array([])
    },
    "ItemKNN": {
        "recommendations": np.array([])
    },
    "TopPop": {
        "recommendations": np.array([])
    },
}}

# Loading predictions
for recommender in recommendations["recommenders"].keys():
    recommendations["recommenders"][recommender]["recommendations"] = np.load(f"recommendations/{recommender}.npy")

config.update(recommendations)

Implement `evaluate_jsd` and then call it from `evaluate_algorithms`. Do not use any global variables.
For each user construct the two popularity distributions (from history and recommendations), normalize them, push through JSD and return the overall mean over users as the result.

In [22]:
def evaluate_jsd(recommendations: np.ndarray, user_histories: list, item_pop_bins: np.ndarray):
    '''
    recommendations - 2D np.ndarray, array containing indices of recommended items for every user;
    user_histories - list of np.ndarray, containing the item indices of user interactions from the training set;
    item_pop_bins - 1D np.ndarray, array that contains the popularity bin number of an item at the respective index;

    returns - float, mean of jensen-shannon divergence over all users;
    '''

    jsd_list = []
    
    for user_history, user_recommendations in zip(user_histories, recommendations):
        hist_pop_counts = np.bincount(item_pop_bins[user_history], minlength=3)
        hist_pop_dist = hist_pop_counts / np.sum(hist_pop_counts)
        
        rec_pop_counts = np.bincount(item_pop_bins[user_recommendations], minlength=3)
        rec_pop_dist = rec_pop_counts / np.sum(rec_pop_counts)
        
        jsd = JSD(hist_pop_dist, rec_pop_dist)
        jsd_list.append(jsd)
    
    return np.mean(jsd_list)


In [23]:
user_histories = [np.array([5, 6]), np.array([7, 8, 9])]
item_pop_bins = np.array([0, 0, 0, 0, 0, 0, 0, 1, 2, 2])
recommendations = np.array([[0, 1, 8, 3, 4], [0, 1, 2, 3, 4]])

jsd = evaluate_jsd(recommendations, user_histories, item_pop_bins)

assert np.isclose(jsd, 0.554015)

`evaluate_algorithms` takes the config as an input and evaluates popularity bias of all algorithms listed in it.

In [24]:
def evaluate_algorithms(config: dict):
    '''
    config - dictionary, provides the run config for the evaluation, use the already provided "config";

    returns - dictionary, result dictionary as provided within the function;
    '''
    result = {
        "recommenders": {
            "SVD": {"jsd": 0},
            "ItemKNN": {"jsd": 0},
            "TopPop": {"jsd": 0}
        }
    }
    for recommender in result["recommenders"].keys():
        recommendations = config["recommenders"][recommender]["recommendations"]
        jsd_value = evaluate_jsd(recommendations, config["user_histories"], config["item_pop_bins"])
        result["recommenders"][recommender]["jsd"] = jsd_value
    return result

In [25]:
result = evaluate_algorithms(config)
assert isinstance(result["recommenders"]["SVD"]["jsd"], float)

In [26]:
# Investigate your results - which algorithm has the lowest bias/divergence?
print(result)

{'recommenders': {'SVD': {'jsd': 0.10694457991259945}, 'ItemKNN': {'jsd': 0.2476013290040527}, 'TopPop': {'jsd': 0.5042004236301939}}}
